In [1]:
%%configure -f
{ "conf":{
          "spark.pyspark.python": "python3",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.type":"native",
          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv",
          "spark.sql.execution.arrow.enabled":"true"
         }
}

[I 2023-03-13 23:34:15,825.825 configure_magic] Magic cell payload received: {"conf": {"spark.pyspark.python": "python3", "spark.pyspark.virtualenv.enabled": "true", "spark.pyspark.virtualenv.type": "native", "spark.pyspark.virtualenv.bin.path": "/usr/bin/virtualenv", "spark.sql.execution.arrow.enabled": "true"}, "proxyUser": "assumed-role_fdp_blitvin-Isengard"}

[I 2023-03-13 23:34:15,826.826 configure_magic] Sending request to update kernel. Please wait while the kernel will be refreshed.


The kernel is successfully refreshed.

In [ ]:
#%pip install pyEX

In [ ]:
#%pip install awswrangler

In [3]:
import http.client

conn = http.client.HTTPSConnection("api2.msci.com")

headers = { 'Authorization': "Basic REPLACE_BASIC_AUTH" }

conn.request("GET", "/index/performance/v1.0/indexes/%7BentityId%7D/close?calc_date=20210405&as_of_date=20210405&distribution_zone=SOME_STRING_VALUE&cumulative=SOME_BOOLEAN_VALUE&index_variant=GRTR&currency=USD&data_frequency=SOME_STRING_VALUE&output=SOME_STRING_VALUE", headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))


In [5]:
vars(res)

In [1]:
import pyEX as p
import requests
import json
import boto3
import pandas as pd
import numpy as np
#import vectorbt as vbt
import matplotlib.pyplot as plt 
import itertools 
import datetime 
import awswrangler as wr
import vectorbt as vbt
import pyspark.sql.functions as py_f
import pyspark.sql.types as py_t
from string import Template
isSandbox = False
if isSandbox:
    token ='Tpk_02dcd2036e7641b880dd4cbb01fa9c67'
    iex_ver = 'sandbox'
else:
    #token ='pk_2e94555e43da4135a6032433c6b18fa5'pk_79d147436c1349f3abbec37591323e52
    token ='pk_79d147436c1349f3abbec37591323e52'
    iex_ver = 'stable'
pyEX_cl = p.Client(api_token=token)
timeframe='max'

/usr/local/lib/python3.7/site-packages/tzlocal/unix.py:177: UserWarning: Can not find any timezone configuration, defaulting to UTC.
  warnings.warn("Can not find any timezone configuration, defaulting to UTC.")


In [3]:
def get_ETF_constituents(symbol,etf_holdings_dates_url,etf_holdings_url,schema):
    if isinstance(symbol, str):
        pass
    else:
        symbol=symbol.values[0][0]
         
    cols= [i.split(" ")[0] for i in schema.split(",")]
    vals= ['' if i.split(" ")[1] == 'string' else 0  for i in schema.split(",") ]
    empty_pd= pd.DataFrame([vals],columns=cols)
    findataprep_dates_url = etf_holdings_dates_url.substitute(symbol=symbol)
    empty_pd['etf_symbol']=f"{symbol} : {findataprep_dates_url} : {etf_holdings_url}"
    try:
        dates = requests.get(findataprep_dates_url)
    except  Exception as l_exc:
        ret_pd=empty_pd
        ret_pd['name']=l_exc 
        ret_pd['title']='exception' 
        return(ret_pd)
    all_date_pd = []
    for one_d in dates.json():  
        try:
            findataprep_url = etf_holdings_url.substitute(symbol=symbol,date=one_d.get('date'))
            one_date_json = requests.get(findataprep_url).json()
            one_date_pd=pd.DataFrame(one_date_json)
            one_date_pd['etf_symbol']=symbol
        except:
            one_date_pd=empty_pd
            one_date_pd['etf_symbol']=symbol    
        all_date_pd=all_date_pd+[one_date_pd] 
    if len(all_date_pd)>0:
        ret_pd = pd.concat(all_date_pd)
    else:
        ret_pd=empty_pd
    return(ret_pd)
#@py_f.pandas_udf("date timestamp, close double, open double, symbol string", py_f.PandasUDFType.GROUPED_MAP)
#@py_f.pandas_udf("symbol string", py_f.PandasUDFType.GROUPED_MAP)
def get_new_data_df(symbol):
    local_sym=str(symbol.values[0][0])
    try:
        hwm_l=highwatermark_pd_bc.value
        columns_to_capture=columns_to_capture_bc.value
        iex_base_url=iex_base_url_bc.value
        sym_pd = hwm_l.query(f"symbol=='{local_sym}'")
        if len(sym_pd)>0:
            max_date = sym_pd['max_date'].iloc[0]
        else:
            max_date = datetime.datetime.now()-datetime.timedelta(days=365*50)
        last_days=(datetime.datetime.now()-max_date).days

        pyEX_cl = p.Client(api_token=iex_token_bc.value)
        final_url = iex_base_url.format(local_sym,iex_token_bc.value,int(last_days))
        #'https://cloud.iexapis.com/stable/stock/{}/chart/max?token={}&chartIEXOnly=true&chartLast={}'
        res = requests.get(final_url)
        ret_pd = pd.DataFrame(json.loads(res.text))#[columns_to_capture]
        ret_pd=ret_pd.query(f"date>'{max_date.strftime('%Y-%m-%d')}'")
        ret_pd['date']=pd.to_datetime(ret_pd['date'])
        print('panda UDF')
        #return(ret_pd[columns_to_capture].query(f"date>'{max_date}'"))
    except:
        ret_pd=pd.DataFrame([{'date': datetime.datetime.now()-datetime.timedelta(days=365*50),
                              'close': None, 
                              'open': None, 
                              'symbol': local_sym}])
    #return(ret_pd[columns_to_capture],max_date,hwm_l,local_sym,sym_pd)
    return(ret_pd[columns_to_capture])
    #return(pd.DataFrame([res.text],columns=['symbol']))
    
class MarketDataGateway():
    def __init__(self,isSandbox=False):
        self.fx_etf_symbols = ['FXA','FXY','FXE','FXB','UUP','FXC','FXF',]
        self.bitcoin_etf_symbols = ['BITO','BTF','XBTF','BITS','GBTC','BITW','BLOK','BLCN','LEGR','SPBC','BITQ','BKCH','DAPP','BTCFX']
        self.bitcoin_equity_proxy_symbols = ['COIN','MSTR']
        self.crypto_mining_symbols = ['RIOT','CAN','HUT','HIVE','MARA','BTCM','BTBT','BITF']
        self.commod_etf_symbols = ['USO','GLD','DBA','DBB','SLV',]
        self.fi_etf_symbols = ['IAGG','AGG','IHY','EMLC','HYG',]
        self.equity_etf_symbols = ['AMJ','AMLP','ARKF','ARKG','ARKK','ARKQ','ARKW','BOTZ','BUG','CGW','CIBR','COPX','CRBN','DRIV','EFV','EMLP','FBT','FDN','FIVG',
                'FIW','FTXG','GDX','GNR','GUNR','HACK','IAT','IBB','ICLN','IEO','IFRA','IGE','IGF','IGM','IGV','IHF','IHI','ITB','ITZ','IWD','IWN','IWS','IYT','JETS','KBWB',
                      'KOMP','KRE','KWEB','LCTU','LIT','MLPA',
                    'MLPX','MOO','NFRA','OIH','PABU','PAVE','PBW','PEJ','PHO','PICK','PPA','QCLN','REET','REMX','ROBO','SCHH','SIL','SKYY','SMH','SOXX','SRVR','TAN','URA','URNM',
                      'VUG','XAR','XBI','XHB','XME','XOP','XSD','QQQ','EWZ','AAXJ','VTI','EWJ','EWA','EWC','MCHI','EWU','EWQ','EWG','FM','EIS']
        self.sectors = ['XLC','XLY','XLP','XLE','XLF','XLV','XLI','XLB','XLRE','XLK','XLU']
        self.bench_symbol=['SPY','QQQ','IVV']
        self.all_symbols =self.fx_etf_symbols+\
                            self.bitcoin_etf_symbols+\
                            self.commod_etf_symbols+\
                            self.bitcoin_equity_proxy_symbols+\
                            self.crypto_mining_symbols+\
                            self.fi_etf_symbols+\
                            self.equity_etf_symbols+\
                            self.bench_symbol+\
                            self.sectors
        self.all_symbols_df = spark.createDataFrame(pd.DataFrame(self.all_symbols,columns=['symbol']))
        #all_symbols
        self.columns_to_capture = ['date','close','open','symbol']
        self.secrets = {
            'IEX':{},
            'alpaca':{ 'token':'alpaca_marketdata_key',
                      'secret':'alpaca_marketdata_secret'}
                       }
        self.data_folder = 's3://fsidatalake/eod_market_data/etf_trend_following/'
        self.parquet_file_out =f"{self.data_folder}/etf_market_data.parquet"
        if isSandbox:
            self.iex_token ='Tpk_02dcd2036e7641b880dd4cbb01fa9c67'
            self.iex_ver = 'sandbox'
        else:
            #self.iex_token ='pk_2e94555e43da4135a6032433c6b18fa5' pk_79d147436c1349f3abbec37591323e52
            self.iex_token ='pk_79d147436c1349f3abbec37591323e52' 
            self.iex_ver = 'stable'
        self.iex_base_url = 'https://cloud.iexapis.com/'+self.iex_ver+'/stock/{}/chart/max?token={}&chartIEXOnly=true&chartLast={}'
        
        self.findataprep_config={}
        self.findataprep_config['etf_holdings_dir']='b753de15443e37f6598e7702811dfad1'
        self.findataprep_config['apikey']='b753de15443e37f6598e7702811dfad1'
        self.findataprep_config['etf_holdings_dates_url']=Template(
            f"https://financialmodelingprep.com/api/v4/etf-holdings/portfolio-date?symbol=$symbol&apikey={self.findataprep_config['apikey']}")
        self.findataprep_config['etf_holdings_url']=Template(
            f"https://financialmodelingprep.com/api/v4/etf-holdings?symbol=$symbol&date=$date&apikey={self.findataprep_config['apikey']}")
        self.findataprep_config['etf_holdings_columns'] = ['cik', 'acceptanceTime', 'date', 'symbol', 'name', 'lei', 'title',
                                                           'cusip', 'isin', 'balance', 'units', 'cur_cd', 'valUsd', 'pctVal',
                                                           'payoffProfile', 'assetCat', 'issuerCat', 'invCountry',
                                                           'isRestrictedSec', 'fairValLevel', 'isCashCollateral',
                                                           'isNonCashCollateral', 'isLoanByFund', 'etf_symbol']
        self.findataprep_config['etf_holdings_column_types']= ['string','string','string',
                                                                'string','string','string','string','string','string',
                                                                'int','string','string','int','float',
                                                                'string','string','string','string','string',
                                                                'string','string','string','string','string',]
        self.findataprep_config['etf_holdings_udf_schema']=",".join([f"{i[0]} {i[1]}" for i in zip( self.findataprep_config['etf_holdings_columns'], self.findataprep_config['etf_holdings_column_types'])])
        self.findataprep_config['etf_holdings_output_file']="s3://fsidatalake/ETF_constituents/etf_holdings.parquet"
        
        
    def set_highwatermarks(self):
        try:
            self.old_df = spark.read.parquet(self.parquet_file_out)
        except:
            self.old_df = spark.createDataFrame(pd.DataFrame([{'symbol':'----','date':datetime.datetime.now()-datetime.timedelta(days=365*50)}]))
        self.highwatermark_pd = self.old_df.groupby('symbol').agg(py_f.max("date").alias("max_date"),
                             py_f.min("date").alias("min_date"),
                             (py_f.max("date")-py_f.min("date")).alias("date_length")
                            ).toPandas()
    def set_new_symbol_df(self):
        self.new_sym_df=self.all_symbols_df.groupby("symbol")\
                        .applyInPandas(get_new_data_df, schema="date timestamp, close double, open double, symbol string")\
                        .where('close is not null')
    def write_new_symbol_df(self):
        self.write_stats=self.new_sym_df.groupby('symbol').agg(py_f.count('symbol').alias('data_count'))
        self.new_sym_df.write\
            .option("header",True)\
            .partitionBy('symbol')\
            .mode("append")\
            .parquet(self.parquet_file_out)
    def rewrite_if_dups(self):
        self.old_df_after_write = spark.read.parquet(self.parquet_file_out)
        self.dup_df = self.old_df_after_write.distinct().groupby('date','symbol').count()
        dup_count=self.dup_df.where('count>1').count()
        if dup_count>0:        
            print(f"found:{dup_count} dups; sample:{self.dup_df.show()}")
            self.old_df_after_write.groupby('date','symbol').agg(py_f.mean('close').alias('close'),py_f.mean('open').alias('open'))\
            .write\
            .option("header",True)\
            .partitionBy('symbol')\
            .mode("overwrite")\
            .parquet(self.parquet_file_out.replace('.parquet','_temp.parquet'))
        else:
            print(f"no dups")
            
    def get_secret(self,secret_name,region_name):
        # Create a Secrets Manager client
        session = boto3.session.Session()
        client = session.client(
            service_name='secretsmanager',
            region_name=region_name
        )

        try:
            get_secret_value_response = client.get_secret_value(
                SecretId=secret_name
            )
        except ClientError as e:
            # For a list of exceptions thrown, see
            # https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
            raise e

        # Decrypts secret using the associated KMS key.
        secret = json.loads(get_secret_value_response['SecretString'])[secret_name]
        return(secret)
        

mdg=MarketDataGateway()
if False:
    mdg.set_highwatermarks()
    # set bc vars
    highwatermark_pd_bc =sc.broadcast(mdg.highwatermark_pd)
    columns_to_capture_bc =sc.broadcast(mdg.columns_to_capture)
    iex_token_bc =sc.broadcast(mdg.iex_token)
    iex_base_url_bc =sc.broadcast(mdg.iex_base_url)
    # set bc vars
    mdg.set_new_symbol_df()
    mdg.write_new_symbol_df()
    print(mdg.write_stats.toPandas().sort_values(['symbol']))
    mdg.rewrite_if_dups()



In [ ]:
etf_holdings_dates_url,etf_holdings_url=mdg.findataprep_config['etf_holdings_dates_url'],mdg.findataprep_config['etf_holdings_url']
schema=mdg.findataprep_config['etf_holdings_udf_schema']
etf_const_df=mdg.all_symbols_df.where(py_f.col('symbol').isin(mdg.bench_symbol)).groupby('symbol').applyInPandas(lambda x: get_ETF_constituents(x,etf_holdings_dates_url,etf_holdings_url,schema), schema=schema )
if True:
    etf_const_df.coalesce(1).write\
            .option("header",True)\
            .partitionBy('symbol')\
            .mode("overwrite")\
            .parquet(mdg.findataprep_config['etf_holdings_output_file'])

In [5]:
if False:
    x=mdg.bench_symbol[2]
    get_ETF_constituents(x,etf_holdings_dates_url,etf_holdings_url)
else:
    etf_const_df.where('title="exception"').show(truncate=False)

+---+--------------+----+------+----+---+-----+-----+----+-------+-----+------+------+------+-------------+--------+---------+----------+---------------+------------+----------------+-------------------+------------+----------+
|cik|acceptanceTime|date|symbol|name|lei|title|cusip|isin|balance|units|cur_cd|valUsd|pctVal|payoffProfile|assetCat|issuerCat|invCountry|isRestrictedSec|fairValLevel|isCashCollateral|isNonCashCollateral|isLoanByFund|etf_symbol|
+---+--------------+----+------+----+---+-----+-----+----+-------+-----+------+------+------+-------------+--------+---------+----------+---------------+------------+----------------+-------------------+------------+----------+
+---+--------------+----+------+----+---+-----+-----+----+-------+-----+------+------+------+-------------+--------+---------+----------+---------------+------------+----------------+-------------------+------------+----------+



In [6]:
etf_const_df.groupBy('etf_symbol','date').count().orderBy('etf_symbol','date').show(1000)

+----------+----------+-----+
|etf_symbol|      date|count|
+----------+----------+-----+
|       IVV|2020-06-30|  394|
|       IVV|2020-07-31|10748|
|       IVV|2020-08-31| 7272|
|       IVV|2020-09-30|   36|
|       IVV|2020-10-31| 4881|
|       IVV|2020-11-30| 2337|
|       IVV|2020-12-31|  288|
|       IVV|2021-01-31|   36|
|       IVV|2021-02-28|11054|
|       IVV|2021-03-31|   53|
|       IVV|2021-04-30|   46|
|       IVV|2021-05-31|   76|
|       IVV|2021-06-30|  539|
|       IVV|2021-07-31|  376|
|       IVV|2021-08-31|  351|
|       IVV|2021-09-30| 2710|
|       IVV|2021-10-31| 2328|
|       IVV|2021-11-30| 5012|
|       IVV|2021-12-31|   76|
|       IVV|2022-01-31|14119|
|       IVV|2022-02-28|  103|
|       IVV|2022-03-31|  852|
|       IVV|2022-04-30| 2228|
|       IVV|2022-05-31|   49|
|       IVV|2022-06-30|   44|
|       IVV|2022-07-31|    8|
|       IVV|2022-08-31|  996|
|       IVV|2022-09-30|  522|
|       IVV|2022-10-31| 5215|
|       IVV|2022-11-30|10291|
|       QQ

In [7]:
print(etf_holdings_dates_url,etf_holdings_url,schema)

<string.Template object at 0x7f29d9959050> <string.Template object at 0x7f29d99e0b10> cik string,acceptanceTime string,date string,symbol string,name string,lei string,title string,cusip string,isin string,balance int,units string,cur_cd string,valUsd int,pctVal float,payoffProfile string,assetCat string,issuerCat string,invCountry string,isRestrictedSec string,fairValLevel string,isCashCollateral string,isNonCashCollateral string,isLoanByFund string,etf_symbol string
